# Experiment 1 (The red/blue/green/yellow house)



In [10]:
import sys
import os

# Get the absolute path of the parent directory.
parent_dir = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

# Add the parent directory to the system path to be able to import modules from 'lib.'
sys.path.append(parent_dir)

In [11]:
from IPython.display import  Markdown as md
import itertools
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy
import numpy as np
import random

import pandas as pd
import pathlib

from lib.memory.DSDM import DSDM
from lib.utils.preprocess import preprocess_text
from lib.utils.utils import generate_atomic_HVs_from_tokens_and_add_them_to_cleanup, generate_chunk_representations_and_save_them_to_memory, fix_seed, load_data, infer, online_learning_with_inference


import torch
import torchhd as thd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F 

# Type checking
from typing import List

In [12]:
#from IPython.display import display, HTML
#display(HTML("<style>.container { width:50% !important; }</style>"))

In [35]:
from ipywidgets import HBox, Output

x = pd.DataFrame([{'a': 1}])
x.to_html().replace("dataframe", "inlineTable")

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>a</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1</td>\n    </tr>\n  </tbody>\n</table>'

In [4]:
# Set experiment number.
experiment_number = 1

# Fix seed.
fix_seed()

# Set device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dim = 2000 # Vector dimension

cleanup = {} # Cleanup memory for saving atomic HVs

initial_trainig_data_path = "../data/experiment-1/initial_training_data.txt"
trainig_data_path = "../data/experiment-1/data.txt"

[ Using Seed :  42  ]


In [5]:
# DSDM hyperparameters
address_size = dim
ema_time_period = 5000  # No. of days in the EMA, i.e., maximum number of save operations to be performed.
learning_rate_update = 0.1
temperature = 0.2

# Create DSDM instances.
memory_unnormalized = DSDM(address_size=address_size, ema_time_period=ema_time_period, learning_rate_update=learning_rate_update, temperature=temperature)
memory_normalized = DSDM(address_size=address_size, ema_time_period=ema_time_period, learning_rate_update=learning_rate_update, temperature=temperature, normalize=True)

memories = {"normalized": memory_normalized, "unnormalized": memory_unnormalized}

In [6]:
# Load initial training data.
lines_raw = load_data(initial_trainig_data_path)

# Preprocess initial training data. 
lines_tokens = []
for line_raw in lines_raw:
    # Account for empty lines.
    if line_raw.rstrip():
        lines_tokens.append(preprocess_text(line_raw))

# Flush cleanup memory.
cleanup = {}

# Define chunk lengths
chunk_lengths = [1, 2, 3]

# Train memories with initial training data.
for sentence_tokens in lines_tokens:
    generate_atomic_HVs_from_tokens_and_add_them_to_cleanup(dim, cleanup, sentence_tokens)
    for _, memory in memories.items():
        # Construct the chunks of each sentence and save them to each memory.
        generate_chunk_representations_and_save_them_to_memory(dim, cleanup, memory, sentence_tokens, chunk_lengths=[1, 2, 3])        

### Inference

In [7]:
inference_sentences = ["The red house.", "The house.", "House.", "The purple house."]
tokens_to_keep_track_of = ['red']

# Get index to get the similarities of the tokens we keep track of for current epoch.
index = list(itertools.product(inference_sentences, tokens_to_keep_track_of))

In [8]:
# Construct dataframes for keeping track of token similarities.
tracked_tokens_sims_dfs = {}
for memory_type, memory in memories.items():
    tracked_tokens_sims_dfs[memory_type] = infer(dim, cleanup, memory, inference_sentences).loc[index]

# Online training.
online_learning_with_inference(experiment_number,
                               dim,
                               cleanup,
                               memories,
                               inference_sentences,
                               index,
                               tracked_tokens_sims_dfs,
                               chunk_lengths)

 <ins>Normalized</ins>

epoch: 0

similarity
sentence          token             
The red house.    red       0.738053
                  the       0.471178
                  house     0.462160
                  blue      0.061105
                  purple    0.028926
                  green    -0.009104
The purple house. the       0.604550
                  house     0.557373
                  red       0.503960
                  blue      0.203475
                  green     0.135177
                  purple    0.031066
The house.        the       0.586857
                  house     0.582263
                  red       0.496501
                  blue      0.198324
                  green     0.141308
                  purple    0.029992
House.            house     0.901664
                  red       0.278354
                  blue      0.236242
                  green     0.184747
                  the       0.114681
                  purple    0.008889

 <ins>Unnormalized</ins>

epoch: 0

...

epoch: 9

similarity
sentence          token             
The red house.    red       0.780835
                  the       0.449741
                  house     0.412174
                  blue      0.042355
                  purple    0.028560
                  green    -0.028162
The purple house. red       0.681802
                  the       0.528619
                  house     0.478212
                  blue      0.104737
                  green     0.033974
                  purple    0.030421
The house.        red       0.677468
                  the       0.522493
                  house     0.490657
                  blue      0.102993
                  green     0.037321
                  purple    0.029999
House.            house     0.843794
                  red       0.429352
                  the       0.217929
                  blue      0.175701
                  green     0.116870
                  purple    0.015095

 <ins>Unnormalized</ins>

epoch: 9

similarity
sentence          token             
The red house.    red       0.629322
                  the       0.542590
                  house     0.532846
                  blue      0.101890
                  purple    0.030565
                  green     0.030301
The purple house. the       0.663671
                  house     0.635529
                  red       0.296864
                  blue      0.205371
                  green     0.148784
                  purple    0.031043
The house.        the       0.657011
                  house     0.655068
                  red       0.267709
                  blue      0.198253
                  green     0.159821
                  purple    0.029976
House.            house     0.949986
                  red       0.206568
                  the       0.153437
                  blue      0.132595
                  green     0.093573
                  purple    0.010022

### Memory state

In [9]:
for memory_type, memory in memories.items():
    display(md(f"### <ins>{memory_type.capitalize()}</ins>"))
    concepts_df = display_and_get_memory_addresses(memory)
    
    concepts_df['memory_concept_str'] = concepts_df['memory_concept'].apply(lambda concept_list: " ".join(concept_list))
    print(concepts_df['memory_concept_str'].values)
    
    # Get cosine similarties of the memory addresses mapping to the same concept.
    tmp_df = pd.DataFrame(concepts_df.groupby('memory_concept_str')['memory_address'].apply(list)).reset_index()
    for i in range(len(tmp_df)):
        address_list = tmp_df['memory_address'][i]
        
        if len(address_list) > 1: 
            stacked_tensor = torch.stack(address_list, dim=0)
            pairwise_similarities = torch.nn.functional.cosine_similarity(stacked_tensor.unsqueeze(1), stacked_tensor.unsqueeze(0), dim=2)
            print(pairwise_similarities)
            # Why are they all identical? 

### <ins>Normalized</ins>

NameError: name 'display_and_get_memory_addresses' is not defined